In [ ]:
import duckdb

In [ ]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [ ]:
df = con.execute("""
                 SELECT *
                 FROM (
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS rn 
                    FROM bronze_z0019
                    WHERE data_ingestao >= '2025-01-11'
                 ) WHERE rn = 1
                 """).fetchdf()
df.head()

In [ ]:
df_final = df.drop(columns=['nome_arquivo','data_ingestao','rn'])
df_final = df_final.rename(columns={"NATBR": "id",
                                   "MAKTX": "nm_produto",
                                   "WERKS": "id_categoria",
                                   "MAINS": "id_fornecedor",
                                   "LABST": "vl_preco"
                                   })
df_final.head()

In [ ]:
df2 = df_final
df2.dtypes


In [ ]:
df2 = df2.astype({
    'id': 'int32',
    'nm_produto': 'string',
    'id_categoria': 'string',
    'id_fornecedor': 'int32',
    'vl_preco': 'float32'
})
df2 = df2.astype({
    'id': int,
    'nm_produto': str,
    'id_categoria': str,
    'id_fornecedor': int,
    'vl_preco': float
})
# df2.head()
df2.dtypes

In [ ]:
con.execute("""
            CREATE TABLE IF NOT EXISTS produtos (
                id bigint, 
                nm_produto text, 
                id_categoria text, 
                id_fornecedor bigint,
                vl_preco float
            )
            """)

In [ ]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [ ]:
df2.head()

In [ ]:
df_resultado = con.execute("select * from produtos").fetchdf()
df_resultado.head()

In [ ]:
con.close()